In [27]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

data = pd.read_csv('C:/Users/Wazir/Downloads/train_kOBLwZA.csv')

In [3]:
train_data.isnull().sum().sort_values(ascending=False)

Outlet_Size                  2410
Item_Weight                  1463
Item_Outlet_Sales               0
Outlet_Type                     0
Outlet_Location_Type            0
Outlet_Establishment_Year       0
Outlet_Identifier               0
Item_MRP                        0
Item_Type                       0
Item_Visibility                 0
Item_Fat_Content                0
Item_Identifier                 0
dtype: int64

In [8]:
train_data.Item_Weight.fillna(train_data.Item_Weight.mean(), inplace = True)
train_data.Outlet_Size.fillna(train_data.Outlet_Size.mode()[0], inplace = True)

In [9]:
train_data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,Medium,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [12]:
train_data.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
Item_Outlet_Sales            float64
dtype: object

In [29]:
import sklearn

In [17]:
import category_encoders as ce

In [18]:
OHE = ce.OneHotEncoder(cols =['Item_Fat_Content',
                              'Item_Type',
                              'Outlet_Identifier',
                              'Outlet_Size',
                              'Outlet_Location_Type',
                              'Outlet_Type'], use_cat_names=True)

train_data = OHE.fit_transform(train_data)

In [19]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(np.array(train_data.Item_MRP).reshape(-1,1))
train_data.Item_MRP = scaler.transform(np.array(train_data.Item_MRP).reshape(-1,1))


In [20]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [21]:
train_X = train_data.drop(columns= ['Item_Identifier', 'Item_Outlet_Sales'])
train_Y = train_data['Item_Outlet_Sales']

In [22]:
train_x, test_x, train_y, test_y = train_test_split(train_X, train_Y, test_size=0.25, random_state= 0)

In [23]:
#Linear Regression
model_LR = LinearRegression()
model_LR.fit(train_x, train_y)

predict_train = model_LR.predict(train_x)
predict_test = model_LR.predict(test_x)

print("RMSE on train data :" , mean_squared_error(train_y, predict_train)**0.5)
print("RMSE on test data :", mean_squared_error(test_y, predict_test)**0.5)

RMSE on train data : 1120.4175764671347
RMSE on test data : 1148.734817781439


In [28]:
data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [44]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
import category_encoders as ce
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline

In [45]:
train_x = data.drop(columns = ['Item_Outlet_Sales'])
train_y = data['Item_Outlet_Sales']

In [49]:
from sklearn.base import BaseEstimator

class OutletTypeEncoder(BaseEstimator):
    def __init__(self):
        pass

    def fit(self, documents, y=None):
        return self

    def transform(self, x):
        x['outlet_grocery_store'] = (x['Outlet_Type'] == 'Grocery Store')*1
        x['outlet_supermarket_3'] = (x['Outlet_Type'] == 'Supermarket Type3')*1
        x['outlet_identifier_OUY027'] = (x['Outlet_Type'] == 'OUT027')*1
        
        return x
    

In [54]:
pre_process = ColumnTransformer(remainder='passthrough',
                                transformers = [('drop_columns', 'drop',
                                                ['Item_Identifier', 'Outlet_Identifier', 'Item_Fat_Content',
                                                 'Item_Type', 'Outlet_Identifier', 'Outlet_Size',
                                                 'Outlet_Location_Type', 'Outlet_Type']),
                                                ('impute_item_weight', SimpleImputer(strategy='mean'),
                                                ['Item_Weight']),
                                                ('scale_data', StandardScaler(),
                                                ['Item_MRP'])])

In [55]:
model_pipeline = Pipeline(steps = [('get_outlet_binary_columns', OutletTypeEncoder()),
                                    ('preprocessing', pre_process),
                                    ('random_forest', RandomForestRegressor(max_depth=10, random_state= 2))])

model_pipeline.fit(train_x, train_y)

model_pipeline.predict(train_x)

C:\Users\Wazir\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


array([4125.52249642,  645.60164729, 2443.67220635, ..., 1287.24545144,
       1410.15236359, 1176.92845236])

In [56]:
#ABALONE SET

In [60]:
dataframe = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/abalone.csv', header= None)
last_ix = len(dataframe.columns) - 1

X,y = dataframe.drop(last_ix, axis =1), dataframe[last_ix]
print(X.shape, y.shape)

(4177, 8) (4177,)


In [67]:
numerical_ix = X.select_dtypes(include=['int64', 'float64']).columns
categorical_ix = X.select_dtypes(include=['object', 'bool']).columns

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

In [71]:
t = [('anything1', OneHotEncoder(), categorical_ix),
     ('anything2', MinMaxScaler(), numerical_ix)]

col_transform = ColumnTransformer(transformers = t)

In [75]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
import numpy as np

In [80]:
model = SVR(kernel ='rbf', gamma='scale', C=100)
pipeline = Pipeline(steps = [('prep', col_transform), ('m', model)])

cv = KFold(n_splits=10, shuffle=True, random_state=1)
scores = cross_val_score(pipeline, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
scores = abs(scores)

print('MAE: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

MAE: 1.465 (0.047)


In [83]:
print('MAE: %.4f (%.5f)' % (np.mean(scores), np.std(scores)))

MAE: 1.4648 (0.04704)


In [89]:
x= np.arange(0,100)

x = x.reshape(-1,2)

x.shape

(50, 2)